In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
import more_itertools as locate
import re

In [7]:
FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest" #sets filepath for initial file/mass list. May not be necessary to keep this way
                                                            #but sets path for list 
FileType="csv" #sets filetype
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv("NewIncompMassList.csv") #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
#print(CurrFile)
MassNumberNames=[]
#######################################


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])
print(len(MassNumberNames))
print(len(MassNumbersList))

47
47


In [8]:
FilePath = "/Users/homefolder/Documents/RapfLab/code/MassSpecData/210711MSCHR/MS" #Second file path for where the actual files are located
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+"Blank"+"*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################
ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]
TotalList=[]
TotalList2=[]
mzlistRepTot=[]
FinalList=[]

 
for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point")
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
             
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])      
                
    
    for i in range(len(ProcessedX)): 
        TotalList.append(ProcessedX[i])
        TotalList2.append(ProcessedY[i])
        mzlistRepTot.append(mzlistRep[i])
        
            
for mass in range(len(MassNumbersList)):
    Matchlist=[]
    List=[]
    searchval = (MassNumbersList[mass])
    ii = np.where(mzlistRepTot == searchval)[0]
    List=str(ii).split(" ")
    str(List).strip()
    res = []
    for i in List:
        m = re.search(r"\b(\d{1,2})\b", i)
        if m:
            res.append(m.group())
    Final=[]
    for i in range(len(res)):
        Final.append((TotalList2[int(res[i])]))
    stdList=[]
    for i in range(len(Final)):
        mean = sum(Final) / len(Final)
        variance = sum([((x - mean) ** 2) for x in Final]) / len(Final)
        std = variance ** 0.5
    stdList.append(std)
    for i in range(len(stdList)):    
        FinalList.append((stdList[i]))

print(FinalList)

        
IndFile = pd.DataFrame(columns = ["mz_list","standard deviation"])
IndFile["mz_list"]=MassNumbersList
IndFile["standard deviation"]=FinalList
IndFile.to_csv("/Users/homefolder/Documents/RapfLab/code/MStest/Processed"+"stdev"+"_blank.csv")

[3730.008249282837, 155.15592312812817, 12555.397378921509, 21.85053563100197, 22.6053829193115, 25422.590173721313, 15969.017013549805, 559.1912288665796, 36321.625, 6.8375611305237, 0.0, 506.929748535155, 2.5499018596878917, 2.5499018596878917, 3.3564220853503275, 253.048614501953, 481.9343881607058, 1.212236762046815, 1.00137257575989, 2739.2759053961618, 1492.5073191373497, 118.20225834846478, 6.0101275444031, 222.83030307292944, 238.17221927642817, 31.7951374053955, 45.06894290447233, 8.67534255981445, 0.998690485954285, 1.333120822906495, 2.11351728439331, 5.305780783060061, 652.2170922756183, 1887.71423339844, 548.9640157084882, 21.39295387268065, 0.546944439411165, 2.765783071517945, 2.3329564636515365, 1.53907573223114, 47.979097843170344, 228.26376378536236, 4.996249198913575, 0.0, 0.0, 0.0, 0.0]
